# What is Porter's Algorithm?

Porter's Algorithm, or the "Porter Stemmer", is an algorithm for stemming English words by removing their suffixes. The original paper can be found [here](http://cl.lingfil.uu.se/~marie/undervisning/textanalys16/porter.pdf). 

The paper itself sites the function of stemming words as useful for information retrieval. Stemming can reduce the complexity and size of a corpus by reducing many different words down to one term. The paper gives an example of this with the words *connect*, *connected*, *connecting*, *connection*, and *connections*. After stemming all of these words, we have reduced 4 words down to just one, *connect*. 

# Calculating the measure of a stem or word

Important notation from the original paper:

$V$ = any arbitrary number of vowels in a row

$C$ = any arbitrary number of consonants in a row

Ex: The string "abcdefg" can be written as v-c-c-c-v-c-c. This can be shortened to $VCVC$ 

One of the most important aspects of Porter's algorithm is what the original paper calls the *measure* of the stem or word. This is essentially counting the amount of times in a stem or word when a $VC$ occurs. If $VC$ occurs $m$ times, then the measure of the word = $m$. The measure of the word becomes important when the algorithm has to decide if a word is already stemmed. Typically a word with a $m < 1$ will not benefit from being stemmed. 

The paper gives the following representation for any given word:


$$[C] \; (VC)^{m} \; [V]$$

where the the $[C]$ and $[V]$ are arbitrary amounts of either consonants or vowels respectively. 

The below function will calculate the measure of a given word or stem. It will then return the above representation of the word, as well as the measure of the word, in a tuple.

In [2]:
vowels = ['a', 'e', 'i', 'o', 'u']

consonants = ['b', 'c', 'd', 'f', 'g', 'h', 'j', 'k', 'l', 'm', 'n', 
              'p', 'q', 'r', 's', 't', 'v', 'w', 'x', 'y', 'z']

In [3]:
def calculate_measure(word):
    ar = list(word.lower())
    VoC = []
    for letter in ar:
        if letter in vowels:
            VoC.append('V')
        else: VoC.append('C')
    
    blocks = [VoC[0]]
    current = ''
    for i in range(len(VoC)):
        current = VoC[i]
        if current != blocks[-1]:
            blocks.append(current)
    
    measure = 0
    for i in range(len(blocks) - 1):
        if (blocks[i] == 'V'):
            if (blocks[i + 1] == 'C'):
                measure += 1
                
    return blocks, measure

In [5]:
print(calculate_measure("trees"))
print(calculate_measure("never"))

(['C', 'V', 'C'], 1)
(['C', 'V', 'C', 'V', 'C'], 2)


The next two cells are helper functions for some of the later steps. 

The function *endswithdoublecons()* checks if the word or stem ends in double consonants. For example, the function would return true for "catt" or "chess" but would return false for "cat" or "ches". 

The next helper function, *endcvc()* checks if the last three letters of the word or stem occur in the order $CVC$. For example, thr function would return true for "plaster" and "caresses" but false for "motoring" and "ponies".

In [4]:
def endswithdoublecons(word):
    if (word[-1] in consonants) & (word[-2] == word[-1]):
        return True

In [5]:
def endcvc(word):
    if (word[-3] in consonants) & (word[-1] in consonants) & (word[-2] in vowels):
        return True

# Getting to the Steps

Now the steps are going to get very formulaic. It probably won't be much use explaining which each step is for. The original paper already does this extremely efficiently. As you'll come to notice, a lot of these steps will require dictionaries of suffix-pairs. This is because each step is checking for a condition, then either removing a suffix, adding a suffix, or both. 

# Step 1a

In [7]:
def step1a(word):
    if word.endswith('sses') or word.endswith('ies'):
        return word[0:len(word) - 2]
    elif word.endswith('s'):
        return word[0:len(word) - 1]
    return word

# Step 1b 

In [8]:
def step1b(word):
    stem= ''
    if word.endswith('eed'):
        stem = word[0:len(word) - 3]
        if calculate_measure(stem)[1] > 0:
            return word[0:len(word) - 1]
        else: return (False, word)
    elif word.endswith('ed'):
        stem = word[0:len(word) - 2]
        if any(vowel in stem for vowel in vowels):
        
            #returns true if word ended in "ed"
            return (True, stem)
        else: return (False, word)
        
        stem = word[0:len(word) - 2]
    elif word.endswith('ing'):
        stem = word[0:len(word) - 3]
        if any(vowel in stem for vowel in vowels):
            #returns true if word ended in "ing"
            return (True, stem)
        else: return (False, word)
    else: return (False, word)

I should add that the below *step1badd()* function isn't an explicit "step" in the algorithm. Rather, it's a function that works off of the results in the previous 1b step. 

In case you didn't notice in the code of Step 1b, the function not only returns a word/stem, but also a boolean value. I used these values to check for conditions on the returned word from this step, so that *step1badd()* can utilize this information. 

Information on this part of the algorithm is on the bottom of page 215 of the original paper. 

In [9]:
def step1badd(word):
    end_result, stem = step1b(word)
    if end_result == True:
        if stem.endswith("at") or stem.endswith("bl") or stem.endswith("iz"):
            return stem + "e"
    
        elif (endswithdoublecons(stem) == True) & (stem[-1] not in ['l', 's', 'z']):
            return stem[0:len(stem) - 1]
    
        elif (calculate_measure(stem)[1] == 1) & (endcvc(stem) == True):
            return stem + "e"
        else: return stem
    return word

# Step 1c

In [10]:
def step1c(word):
    if any(vowel in word for vowel in vowels) & (word[-1] == 'y'):
        return (word[0:len(word)-1] + 'i')
    else: return word

# Step 2

In [12]:
step2dict = {
    "ational":"ate","tional":"tion","enci":"ence",
    "anci":"ance","izer":"ize","abli":"able",
    "alli":"al","entli":"ent","eli":"e",
    "ousli":"ous","ization":"ize","ation":"ate",
    "ator":"ate","alism":"al","iveness":"ive",
    "fulness":"ful","ousness":"ous","aliti":"al",
    "iviti":"ive","biliti":"ble"
}

In [33]:
def step2(word):
    if calculate_measure(word)[1] > 0:
        for suffix in step2dict:
            if word.endswith(suffix):
                return word[0:len(word) - len(suffix)] + step2dict[suffix]
    return word

# Step 3

In [14]:
step3dict = {
    "icate":"ic",
    "ative":"",
    "alize":"al",
    "iciti":"ic",
    "ical":"ic",
    "ful":"",
    "ness":""
}

In [32]:
def step3(word):
    if calculate_measure(word)[1] > 0:
        for suffix in step3dict:
            if word.endswith(suffix):
                return word[0:len(word) - len(suffix)] + step3dict[suffix]
    return word

# Step 4

In [17]:
step4dict = {
    "al":"", "ance":"", "ence":"",
    "er":"", "ic":"", "able":"",
    "ible":"", "ant":"", "ement":"",
    "ment":"", "ent":"", "sion":"",
    "tion":"", "ou":"", "ism":"",
    "ate":"", "iti":"", "ous":"",
    "ive":"", "ize":""
}

In [31]:
def step4(word):
    if calculate_measure(word)[1] > 1:
        for suffix in step4dict:
            if word.endswith(suffix):
                return word[0:len(word) - len(suffix)] + step4dict[suffix]
    return word

# Step 5a 

In [20]:
def step5a(word):
    if (calculate_measure(word)[1] > 1) & word.endswith('e'):
        return word[0:len(word) - 1]
    return word

# Step 5b

In [21]:
def step5b(word):
    if (calculate_measure(word)[1] > 1) & (endswithdoublecons(word) == True) & word.endswith("l"):
        return word[0:len(word) - 1]
    return word

# Combining all the steps

It looks a little messy, but the below function is simply running through each function and returning the resulting word or stem. This works because I wrote all of the above functions to return the original word if none of the conditions apply. This means that any word can safely run through each function without being chopped up incorrectly. 

In [23]:
def stem_word(word):
    return(step5b(
        step5a(
            step4(
                step3(
                    step2(
                        step1c(
                            step1badd(
                                step1a(word)))))))))

# Testing the Algorithm

Let's now test the algorithm on a paragraph taken from James Joyce's *Ulysses*. 

In [35]:
example_string = "Solemnly he came forward and mounted the round gunrest. He faced about and blessed gravely thrice the tower, the surrounding land and the awaking mountains. Then, catching sight of Stephen Dedalus, he bent towards him and made rapid crosses in the air, gurgling in his throat and shaking his head. Stephen Dedalus, displeased and sleepy, leaned his arms on the top of the staircase and looked coldly at the shaking gurgling face that blessed him, equine in its length, and at the light untonsured hair, grained and hued like pale oak."
example_string = example_string.replace(".", "")
arr = example_string.split()
print(example_string + "\n")

stemmed_arr = []

for word in arr:
    if len(word) < 5:
        stemmed_arr.append(word)
        
    else: stemmed_arr.append(stem_word(word))


print(" ".join(stemmed_arr))

Solemnly he came forward and mounted the round gunrest He faced about and blessed gravely thrice the tower, the surrounding land and the awaking mountains Then, catching sight of Stephen Dedalus, he bent towards him and made rapid crosses in the air, gurgling in his throat and shaking his head Stephen Dedalus, displeased and sleepy, leaned his arms on the top of the staircase and looked coldly at the shaking gurgling face that blessed him, equine in its length, and at the light untonsured hair, grained and hued like pale oak

Solemnli he came forward and mount the round gunrest He face about and bless grave thrice the tower, the surround land and the awak mountain Then, catch sight of Stephen Dedalus, he bent toward him and made rapid cross in the air, gurgl in his throat and shake his head Stephen Dedalus, displeas and sleepy, lean his arms on the top of the staircas and look coldli at the shake gurgl face that bless him, equin in its length, and at the light untonsur hair, grain and 

As shown from the results above, Porter's algorithm isn't perfect, but it does an incredible job stemming words.